In [ ]:

import os
import pandas as pd
import openai
import time
from sklearn.metrics import classification_report

openai.api_key = ''

folder = r"Data\Multiple_Neg_Data"
global results2
print(folder)

In [69]:
def process_missing_keys(prompt, sentences,missingkeys,results,batch,batch_indices,batchnumber):
    print("i am here in process_missing_keys")
    m={}
    for key,indexlist in missingkeys.items():
        for index in indexlist:
            m[index]=sentences[index]
    
    batch_indices=list(m.keys())
    batch=list(m.values())
    print(batch_indices)
    print(batch)


    def process_batch(prompt, batch,batch_indices,batchnumber,results,missingkeys):
       
        print("results in start process_missing_keys function ", results)

        prompt += "\n".join([f"{i}. Classify the sentence '{batch[i]}' as Multiple Negation or Not Multiple Negation in one word which is a concept of AAVE English and keep the numbering given at the start of the prompt intact in the response."
                            for i in range(len(batch))])
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            )

            responses_map = {}
            responses = [res.strip() for res in response.choices[0].message['content'].strip().split('\n')]
            print("Actual responses in process_missing_keys ", responses)

            lineindex=0
            for line in responses:
                if ". " in line:
                    try:
                        indexr, value = line.split(". ", 1)
                        indexr = int(indexr)
                        responses_map[batch_indices[indexr]] = value
                    except ValueError:
                        print(f"Malformed response line : '{line}'")
                        print("linindex:  ",lineindex)
                        if 'M' in line or 'm' in line or 'N' in line or 'n' in line:
                            responses_map[batch_indices[lineindex]]=line
                    lineindex+=1
                else:
                    print(f"Line does not contain expected delimiter: '{line}'")

            for i, idx in enumerate(batch_indices):
                if idx in responses_map:
                    results[idx] = responses_map[idx]
                    print("************************************",batchnumber,missingkeys,idx,results[idx])
                    del missingkeys[batchnumber]
                # else:
                #     if batchnumber not in missingkeys:
                #         missingkeys[batchnumber] = []
                #     missingkeys[batchnumber].append(idx)

            print("Responses Map in process_missing_keys ", results)
            print("Missing Keys in process_missing_keys", missingkeys)
            
           

        except openai.error.RateLimitError:
            print("Rate limit exceeded. Waiting to retry...")
            time.sleep(80)  # Wait for 80 seconds before retrying
        # except Exception as e:
        #     print(f"Error processing batch: {e}")
        return results,missingkeys
    return process_batch(batch,batch_indices,batchnumber,results,missingkeys)
        

In [70]:
import pandas as pd

def generate_prompt(train_data):
    """
    Generates a balanced prompt string with 3 'Multiple Negation' and 3 'No Negation' examples.

    Parameters:
    train_data (pd.DataFrame): DataFrame containing 'Text' and 'Mneg' columns.

    Returns:
    str: The generated balanced prompt string.
    """
    # Separate the data into two categories: Multiple Negation and No Negation
    multiple_negation = train_data[train_data['Mneg'] == 1]
    no_negation = train_data[train_data['Mneg'] == 0]

    # Sample 3 examples from both categories to ensure balance
    sampled_multiple_negation = multiple_negation.sample(3, random_state=42)
    sampled_no_negation = no_negation.sample(3, random_state=42)

    # Combine the two sampled sets
    balanced_data = pd.concat([sampled_multiple_negation, sampled_no_negation])

    examples = []
    for _, row in balanced_data.iterrows():
        sentence = row['Text']
        label = "Multiple Negation" if row['Mneg'] == 1 else "No Negation"
        examples.append(f"Sentence: {sentence}\nLabel: {label}\n")

    text = "I have given a few classified train examples:\n"
    prompt = text + "\n".join(examples)

    print("Generated prompt:\n", prompt)
    return prompt




In [71]:
def process_batch(prompt, sentences, batch, batch_indices, batchnumber):
    global results2
    missingkeys1={}
    prompt += "\n".join([f"{i}. Classify the sentence '{batch[i]}' as Multiple Negation or Not Multiple Negation in one word which is a concept of AAVE English and keep the numbering given at the start of the prompt intact in the response."
                            for i in range(len(batch))])
    # print("promt hereee  ",prompt)
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            
            frequency_penalty=0.5,
            presence_penalty=0.3
        )
        responses_map = {}
        responses = [res.strip() for res in response.choices[0].message['content'].strip().split('\n')]
        print("Actual responses**********: ", responses)
        

        lineindex=0
        for line in responses:
            if ". " in line:
                try:
                    indexr, value = line.split(". ", 1)
                    indexr = int(indexr)
                    responses_map[batch_indices[indexr]] = value
                except ValueError:
                    print(f"Malformed response line : '{line}'")
                    print("linindex:  ",lineindex)
                    if 'M' in line or 'm' in line or 'N' in line or 'n' in line:
                        responses_map[batch_indices[lineindex]]=line
                lineindex+=1
            else:
                print(f"Line does not contain expected delimiter: '{line}'")
        # print("hererre testtt",responses_map)          
        for i, idx in enumerate(batch_indices):
            if idx in responses_map:
                results2[idx] = responses_map[idx]
            else:
                if batchnumber not in missingkeys1:
                    missingkeys1[batchnumber] = []
                missingkeys1[batchnumber].append(idx)

        # print("Responses Map:", responses_map)
        print("Missing Keys:", missingkeys1)
        print("resultss up",results2)

        if missingkeys1:
            print("did i come here")
            
            newresults,newmissingkeys=process_missing_keys(prompt, sentences,missingkeys1,results2,batch,batch_indices,batchnumber)
        # time.sleep(80)      
            missingkeys1=newmissingkeys
            results2=newresults
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Waiting to retry...")
        time.sleep(80)  # Wait for 80 seconds before retrying
    # except Exception as e:
    #     print(f"Error processing batch: {e}")
        time.sleep(80)  # Wait for 80 seconds before retrying
    

    return results2,missingkeys1

In [72]:
# Function to classify sentences in batches
def classify_sentences_batch(prompt, sentences, batch_size=5):
    global results2
    results2 = {i: None for i in range(len(sentences))}  # Initialize results dictionary with None values
    # Process sentences in batches
    batchnumber = 1
    for i in range(0, len(sentences), batch_size):
        batch_indices = list(range(i, min(i + batch_size, len(sentences))))
        localresults,localmissingkeys=process_batch(prompt, sentences,[sentences[idx] for idx in batch_indices], batch_indices, batchnumber)
        
        print("batch indicies",batch_indices)
        print("localreuslts  ",localresults)
        batchnumber += 1
    print("results downnnnnnn******************",results2)
    return results2


In [73]:
import re
def gatherResults(test_data,test_sentences,results):
    print("#################################################################  hcdjcnd")
    fold_predictions = []
    fold_true_labels = []
    for idx in range(len(test_sentences)):
            # Access prediction directly from the results list
            prediction = results[idx] if idx < len(results) else None

            if prediction:  # Check if prediction is not None
                prediction=re.sub(r'^[^a-zA-Z]+', '', prediction)

                prediction = prediction.lower()
                print("prediction******************************* ",prediction)
            if prediction is None:
                # Handle None values
                numeric_prediction = 0  # Assign a default numeric value (e.g., 0) for None predictions
            else:
                # Convert the prediction to numeric: 1 for "habitual be", -1 for "non-habitual be"
                if  prediction.startswith('m'):
                    numeric_prediction = 1
                elif prediction.startswith('n'):
                    numeric_prediction = 0
                else:
                    numeric_prediction = '*'  # Handle unexpected cases

            # Get the true label
            true_label = 1 if test_data.loc[idx, 'Mneg'] == 1 else 0

            fold_predictions.append(numeric_prediction)
            fold_true_labels.append(true_label)

            # print(f"Sentence {idx}: '{test_sentences[idx]}' -> Classification: {numeric_prediction} (True: {true_label})")
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^",fold_predictions)
    return fold_predictions,fold_true_labels
            


In [74]:
def interleave_labels(positive_data, negative_data):
    """
    Interleave sentences with label `1` and `-1` as much as possible and append remaining.
    """
    interleaved_data = []

    pos_len = len(positive_data)
    neg_len = len(negative_data)
    min_len = min(pos_len, neg_len)

    # Interleave `1`s and `-1`s one by one
    for i in range(min_len):
        interleaved_data.append(positive_data.iloc[i].to_dict())  # Convert row to dictionary
        interleaved_data.append(negative_data.iloc[i].to_dict())  # Convert row to dictionary

    # Append the rest of the remaining data
    if pos_len > neg_len:
        interleaved_data.extend(positive_data.iloc[min_len:].to_dict('records'))
    else:
        interleaved_data.extend(negative_data.iloc[min_len:].to_dict('records'))

    # Convert to DataFrame using the column names explicitly
    interleaved_df = pd.DataFrame(interleaved_data, columns=["Text", "Mneg"])
    return interleaved_df


In [75]:
#main code
from sklearn.utils import shuffle

completeresults=[]
data_file=r"C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Multiple_Neg_Data\test multiple negation.txt"
results_folder=r"C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Results_Data\few_shot\M_Neg_Results"
train_data_file=r"C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Multiple_Neg_Data\train multiple negation.txt"
def main():
    # Ensure the results folder exists
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # Load the single text file
    print(f"Processing {data_file}")

    # Read the data from the text file without headers
    data = pd.read_csv(data_file, sep=',', header=None,names=["Text","Mneg"])
    train_data = pd.read_csv(train_data_file, sep=',', header=None,names=["Text","Mneg"])
    # Prepare the sentences and true labels
    test_sentences = data['Text'].tolist()  # First column
    true_labels = data['Mneg'].tolist()     # Second column
    prompt = generate_prompt(train_data)  

####################################################################################################################################
    #--new code added shuffling-----------------------------------------
    # data = shuffle(data, random_state=42)  # Shuffle data with a fixed random seed for reproducibility

   
    # test_sentences = data['Text'].tolist()  # First column
    # true_labels = data['Mneg'].tolist() 
#---------------------------------------------------------------------------------------------

#all 1;s and -1s together-------------------------------------------------
    # positive_data = data[data['Mneg'] == 1]
    # negative_data = data[data['Mneg'] == 0]

    # # Concatenate to get all 1's followed by all -1's
    # grouped_data = pd.concat([positive_data, negative_data], ignore_index=True)

    # # Save the grouped data for verification
   
    # data=grouped_data
    # test_sentences = grouped_data['Text'].tolist()
    # true_labels = grouped_data['Mneg'].tolist()
    # print(len(test_sentences))
#---------------------------------------------------------------------------


#------------------------------------------------------------------------------------------------
# Group all sentences with label 1 and -1 separately
    positive_data = data[data['Mneg'] == 1].reset_index(drop=True)
    negative_data = data[data['Mneg'] == 0].reset_index(drop=True)

    # Interleave sentences with label 1 and -1
    interleaved_data = interleave_labels(positive_data, negative_data)

    # Save the interleaved data for verification
    interleaved_data_file = os.path.join(results_folder, 'interleaved_test_multiple_negation.csv')
    interleaved_data.to_csv(interleaved_data_file, index=False)
    print(f"Interleaved data saved to {interleaved_data_file}")
    data=interleaved_data
    # Prepare the sentences and true labels after interleaving
    test_sentences = interleaved_data['Text'].tolist()
    true_labels = interleaved_data['Mneg'].tolist()

#------------------------------------------------------------------------------------------------

########################################################################################################################################




  
    # Classify sentences in batches
    results = classify_sentences_batch(prompt, test_sentences, batch_size=5)
    
    # # Gather predictions and true labels
    fold_predictions, fold_true_labels = gatherResults(data, test_sentences, results)
    print(test_sentences)
    # Save results to a CSV file
    output_df = pd.DataFrame({
        'Sentence': test_sentences,
        'True_Label': fold_true_labels,
        'Prediction': fold_predictions
    })
    # output_file = os.path.join(results_folder, 'few_shot_predictions_multiple_neg.csv')
    # output_file = os.path.join(results_folder, 'few_shot_random_shuffle_predictions_multiple_neg.csv')

    # output_file = os.path.join(results_folder, 'few_shot_New_shuffled(0_and_1_together_predictions_multiple_neg.csv')
    output_file = os.path.join(results_folder, 'few_shot_New_shuffled_interleaved_predictions_multiple_neg.csv')

    output_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

    print("Complete prediction set:", len(fold_predictions), " True labels:", len(fold_true_labels))
    return fold_predictions, fold_true_labels
    return [],[]

In [76]:
#main run
all_predictions=[]
all_true_labels=[]
# all_predictions
all_predictions,all_true_labels=main()


Processing C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Multiple_Neg_Data\test multiple negation.txt
Generated prompt:
 I have given a few classified train examples:
Sentence: Well; we--just to be honest with you; I believe too many people is really scared to come over here on McLaurin Street; but if you come over here on McLaurin Street; you don't feel nothing
Label: Multiple Negation

Sentence: I reenlist for 24 months; but I reenlist after I come back out and couldn't get no job.
Label: Multiple Negation

Sentence: But we don't have no special time for them.
Label: Multiple Negation

Sentence: There was no such things as a middle school at time.
Label: No Negation

Sentence: No fast food; nothing like that.
Label: No Negation

Sentence: My dad's mother name was Belle Johnson from Cruger; Mississippi.
Label: No Negation

Interleaved data saved to C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude

In [77]:
print(len(all_predictions))
print(len(all_true_labels))

198
198


In [78]:
# print(all_predictions)
i=0
for index,pred in enumerate(all_predictions):
    if pred=='*':
        print(index, pred)
        i+=1
print(i)

0


In [79]:
print(classification_report(all_true_labels, all_predictions, target_names=['Non Multiple Neg (label 0)', 'Multiple Neg (label 1)']))


                            precision    recall  f1-score   support

Non Multiple Neg (label 0)       0.98      0.82      0.89       162
    Multiple Neg (label 1)       0.53      0.92      0.67        36

                  accuracy                           0.84       198
                 macro avg       0.76      0.87      0.78       198
              weighted avg       0.90      0.84      0.85       198



In [ ]:
completeresults